In [ ]:
!pip install numpy tensorflow scikit-learn matplotlib seaborn opencv-python pillow


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import confusion_matrix, classification_report

import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline


In [ ]:
import os
import shutil
import random
from glob import glob
os.chdir('/Users/dianhaoli/Documents/data/')

SRC_DIR = 'train'

valid_ratio = 0.1
test_ratio = 0.1

for split in ['valid', 'test']:
    for label in os.listdir(SRC_DIR):
        os.makedirs(os.path.join(split, label), exist_ok=True)

for label in os.listdir(SRC_DIR):
    files = glob(os.path.join(SRC_DIR, label, '*.jpg'))
    random.shuffle(files)

    valid_end = int(len(files) * valid_ratio)
    test_end  = valid_end + int(len(files) * test_ratio)

    for f in files[:valid_end]:
        shutil.move(f, os.path.join('valid', label))

    for f in files[valid_end:test_end]:
        shutil.move(f, os.path.join('test', label))

print("Created 'valid' and 'test' folders inside /Users/dianhaoli/Documents/data/")

In [ ]:

train_newdata = ImageDataGenerator(
    rescale=1./255,           
    rotation_range=30,        
    width_shift_range=0.1,   
    height_shift_range=0.1,  
    shear_range=0.1,          
    zoom_range=0.25,          
    horizontal_flip=True,    
    fill_mode='nearest'      
)

valid_newdata = ImageDataGenerator(rescale=1./255)
test_newdata  = ImageDataGenerator(rescale=1./255)


train_generator = train_newdata.flow_from_directory(
    'train',
    target_size=(64, 64),    
    batch_size=32,
    class_mode='categorical'  
)

valid_generator = valid_newdata.flow_from_directory(
    'valid',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_newdata.flow_from_directory(
    'test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False

)


In [ ]:
x_batch, y_batch = next(train_generator)
labels = list(train_generator.class_indices.keys())

plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_batch[i])
    plt.axis('off')
    plt.title(f"Label: {labels[np.argmax(y_batch[i])]}")
plt.show()


In [ ]:
model = Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(29, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator
)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history.get('val_accuracy', []), label='val_acc')
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history.get('val_loss', []), label='val_loss')
plt.legend()
plt.xlabel('epoch')
plt.show()
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = model.predict(test_generator)
y_true = test_generator.classes
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=list(test_generator.class_indices.keys()))
disp.plot(xticks_rotation='vertical')



In [ ]:
model.save('asl_model_v1.keras')


In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image

MODEL_PATH = "/Users/dianhaoli/asl_fingerspell/asl_model_v1.keras"   
IMG_SIZE = (64, 64)                

try:
    model = load_model(MODEL_PATH)
    print("Model loaded successfully")
    print("Model input shape:", model.input_shape)
    print("Model output shape:", model.output_shape)
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

class_labels = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
    'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
    'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space'
]
def predict_asl(img):
    if model is None:
        return {"Error: model not loaded": 1.0}
    if img is None:
        return {"Error: no image": 1.0}

    try:
        expected_channels = model.input_shape[-1]
        if expected_channels == 1:
            img = img.convert("L")   
        else:
            img = img.convert("RGB") 
        img = img.resize(IMG_SIZE)
        arr = image.img_to_array(img)
        arr = np.expand_dims(arr, axis=0)
        arr = arr.astype("float32")
        if arr.max() > 1.0:
            arr /= 255.0  
        preds = model.predict(arr, verbose=0)
        probs = preds[0]
        if probs.sum() < 0.99 or probs.sum() > 1.01:
            exp = np.exp(probs - np.max(probs))
            probs = exp / np.sum(exp)
        top_indices = np.argsort(probs)[-3:][::-1]
        results = {class_labels[i]: float(probs[i]) for i in top_indices}
        return results
    except Exception as e:
        print("Prediction error:", e)
        return {"Error": 1.0, "Message": str(e)}
demo = gr.Interface(
    fn=predict_asl,
    inputs=gr.Image(
        sources=["webcam", "upload"],
        type="pil",
        label="Take or upload an ASL hand sign"
    ),
    outputs=gr.Label(
        num_top_classes=3,
        label="Predicted ASL Letter"
    ),
    title="ASL Hand Sign Classifier",
    description="Take or upload a photo of an ASL hand sign to predict the letter.",
    theme=gr.themes.Soft()
)

if __name__ == "__main__":
    demo.launch(
        share=True,         
        inbrowser=True,   
        debug=True,         
        show_error=True
    )
